In [ ]:
import sys, os

ROOT_PATH = None # Insert the PATH to the directory of this notebook here
sys.path.append(ROOT_PATH)

if ROOT_PATH is None:
    raise Exception("Please specify the ROOT_PATH")

import experiment
import tensorflow as tf
import numpy as np

import imp
experiment = imp.reload(experiment)
run_experiment = experiment.run_experiment

exp_prefix = 'penntree'

seqlen = 45
n_seqs = 1957
n_emissions = 20
em_lr = 0.01
n_gt_iter = 10
nl_dict = {3: [1, 2, 3], 5: [2, 3, 5], 10: [2, 3, 5, 10]}

experiment_directories = dict()
failed_exps = []
for n_hidden_states in nl_dict.keys():
    for l in nl_dict[n_hidden_states]:
        for i in range(n_gt_iter):
        
            exp_params = {'dataset_ident': 'penntree_tag',
                  'dataset_params': {'len_cut': seqlen},  
                  'train_perc': 0.5,
                  'standard_params' : {'n_hidden_states': n_hidden_states, 
                                       'em_iter': 40
                                      },
                  'standard_log_config': {'gamma_after_estep': False, 
                                          'gamma_after_mstep': False, 
                                          'samples_after_estep': True, 
                                          'samples_after_mstep': True, 
                                         },
                  'dense_params': {'verbose': False, 
                                   'em_iter': 40,
                                   'n_hidden_states': n_hidden_states, 
                                   'mstep_config': {'l_uz':l, 
                                                    'l_vw': l, 
                                                    'scaling': l*n_emissions*n_hidden_states*seqlen*n_seqs,
                                                    'trainables': 'uvwzz0',
                                                    'em_lr': em_lr,
                                                    'initializer': tf.initializers.random_normal(0., 1.),
                                                    'em_optimizer': tf.train.AdamOptimizer(em_lr),
                                                    'em_epochs': 25,
                                                    'cooc_epochs': 10000,
                                                    'cooc_optimizer': tf.train.AdamOptimizer(0.001),
                                                   }
                                  },
                  'dense_log_config': {'gamma_after_estep': False, 
                                       'gamma_after_mstep': False, 
                                       'samples_after_estep': True, 
                                       'samples_after_mstep': True, 
                                      },
                  'dense_opt_schemes': ('em','cooc'),
                 'compare_to_fair_standard': True,
                 'fair_standard_log_config': {'gamma_after_estep': False, 
                                              'gamma_after_mstep': False, 
                                              'samples_after_estep': True,
                                              'samples_after_mstep': True, 
                                             }
                 }
        
            try:
                directory = run_experiment('dataset_sequences', '%s/penntree_i=%d_n=%d_m=%d_l=%d_' % (exp_prefix, i, n_hidden_states, n_emissions, l),
                           exp_params)

                experiment_directories[(i, n_hidden_states, l)] = directory
            except Exception as e:
                print(e)
                failed_exps.append((i, n_hidden_states, l))
                    
np.save('%s%s/experiment_directories' % (ROOT_PATH, exp_prefix), experiment_directories)                    